In [1]:
import pandas as pd
import numpy as np
import sys
import os
import random
from math import sqrt
sys.path.append('../..')
from modules import utils
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

2023-07-06 21:50:48.416038: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-06 21:50:48.485158: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-06 21:50:48.487163: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-06 21:50:48.487172: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.random.set_seed(SEED)

#### The data

In [3]:
jinja_df = pd.read_csv('../data/jinja_data.csv', parse_dates=['timestamp'])
jinja_df.head()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
0,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
1,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 01:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
2,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 02:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
3,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 03:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23
4,"Jinja Main Street, Jinja",0.437337,33.211051,Jinja,2021-09-01 04:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6142,689753,aq_23


In [4]:
latitudes = jinja_df['latitude'].unique()
longitudes = jinja_df['longitude'].unique()
device_ids = jinja_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

(10, 10, 10)

In [5]:
final_df = pd.DataFrame()
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value']
for i, device_id in enumerate(device_ids):
    device_df = utils.get_device_data(jinja_df, device_id, cols)
    processed_df = utils.preprocessing(device_df)
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,time,latitude,longitude,pm2_5
0,452909.0,0.437337,33.211051,12.2844
1,452910.0,0.437337,33.211051,11.6507
2,452911.0,0.437337,33.211051,22.3980
3,452912.0,0.437337,33.211051,17.4937
4,452913.0,0.437337,33.211051,25.1622


#### Model training and validation

In [6]:
def ffnn(X_train, y_train, epochs=1000, optimizer='RMSProp', dropout=0.2):
    model = tf.keras.Sequential() 
    model.add(Input(shape=(X_train.shape[1],), name='Input-Layer')) 
#     model.add(Dropout(rate=dropout))
    model.add(Dense(128, activation='relu', name='Hidden-Layer1'))
#     model.add(Dropout(rate=dropout))
    model.add(Dense(32, activation='relu', name='Hidden-Layer2'))
#     model.add(Dropout(rate=dropout))
    model.add(Dense(1, activation='linear', name='Output-Layer')) 

    model.compile(optimizer=optimizer, # default='rmsprop', an algorithm to be used in backpropagation
                  loss=tf.keras.losses.MeanSquaredError(),
                 )
    model.fit(X_train, 
          y_train,
          batch_size=32,
          epochs=epochs, # default=1, Number of epochs to train the model
          callbacks=None,
          validation_split=0.2, 
         )
    return model

In [7]:
# def scale_data(X):
#     scaler = MinMaxScaler()
#     X_scaled = X.copy()
#     X_scaled[:, 0] = scaler.fit_transform(X[:, 0].reshape(-1, 1)).flatten()
#     return X_scaled

#### delete from here

In [8]:
# idx = 0
# device_indices = final_df[final_df.latitude==latitudes[idx]].index
# device_df = jinja_df[jinja_df.device_number == device_ids[idx]]
# assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
# test_df = final_df.loc[device_indices]
# test_df.head()

In [9]:
# train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
# assert(len(train_df.longitude.unique()) == len(longitudes)-1)
# assert len(final_df) == len(test_df) + len(train_df)
# train_df.head()

In [10]:
# X_train = train_df.iloc[:, 0:-1]
# y_train = train_df.iloc[:, -1]
# X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)

# X_test = test_df.iloc[:, 0:-1]
# y_test = test_df.iloc[:, -1]
# X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
# X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [11]:
# X_train[0]

In [12]:
# X_train_scaled = scale_data(X_train)
# X_train_scaled

In [13]:
# model = ffnn(X_train, y_train, 500, 'RMSProp')

In [14]:
# y_pred = model.predict(X_test) 
# y_pred.shape, y_test.reshape(-1, 1).shape

In [15]:
# np.min(y_pred), np.mean(y_pred), np.max(y_pred)

In [16]:
# mean_squared_error(y_test, y_pred)

In [17]:
# y_train_pred = model.predict(X_train)
# mean_squared_error(y_train, y_train_pred)

#### end here

In [18]:
def cross_validation(final_df, idx):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = jinja_df[jinja_df.device_number == device_ids[idx]]
    assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
    assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
    assert(len(train_df.longitude.unique()) == len(longitudes)-1)
    assert len(final_df) == len(test_df) + len(train_df)
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
    X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)
#     X_train_scaled = scale_data(X_train)
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
    X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
#     X_test_scaled = scale_data(X_test)
    
    model = ffnn(X_train, y_train, optimizer='Adam', dropout=0.8)
    y_pred = model.predict(X_test) 
    
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    return rmse, mape

In [19]:
rmse_list, mape_list = [], []
for i in range(len(latitudes)):
    rmse, mape = cross_validation(final_df, i)
    rmse_list.append(rmse)
    mape_list.append(mape)
    print(f'{device_ids[i]} successful')

Epoch 1/1000


2023-07-06 21:50:50.523981: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-07-06 21:50:50.524003: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: PL1207-PRO.paris.inria.fr
2023-07-06 21:50:50.524006: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: PL1207-PRO.paris.inria.fr
2023-07-06 21:50:50.524087: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.125.6
2023-07-06 21:50:50.524099: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  525.116.04  Release Build  (dvs-builder@U16-I3-A16-3-3)  Thu Apr 27 18:11:06 UTC 2023
GCC version:  gcc 

289/289 [==============================] - 1s 1ms/step - loss: 14429381.0000 - val_loss: 695.1968
Epoch 2/1000
289/289 [==============================] - 0s 934us/step - loss: 440.1732 - val_loss: 630.7939
Epoch 3/1000
289/289 [==============================] - 0s 828us/step - loss: 448.7069 - val_loss: 714.0650
Epoch 4/1000
289/289 [==============================] - 0s 837us/step - loss: 488.5003 - val_loss: 635.2147
Epoch 5/1000
289/289 [==============================] - 0s 710us/step - loss: 472.7919 - val_loss: 844.3975
Epoch 6/1000
289/289 [==============================] - 0s 750us/step - loss: 530.0157 - val_loss: 988.3795
Epoch 7/1000
289/289 [==============================] - 0s 806us/step - loss: 563.4109 - val_loss: 709.6088
Epoch 8/1000
289/289 [==============================] - 0s 752us/step - loss: 611.3159 - val_loss: 692.2584
Epoch 9/1000
289/289 [==============================] - 0s 793us/step - loss: 601.9983 - val_loss: 868.5683
Epoch 10/1000
289/289 [===============

289/289 [==============================] - 0s 743us/step - loss: 27155.5254 - val_loss: 637.7709
Epoch 75/1000
289/289 [==============================] - 0s 798us/step - loss: 21555.9746 - val_loss: 15788.6992
Epoch 76/1000
289/289 [==============================] - 0s 724us/step - loss: 23540.2852 - val_loss: 2550.6472
Epoch 77/1000
289/289 [==============================] - 0s 916us/step - loss: 22780.8262 - val_loss: 31565.1992
Epoch 78/1000
289/289 [==============================] - 0s 747us/step - loss: 19399.4590 - val_loss: 18866.4238
Epoch 79/1000
289/289 [==============================] - 0s 720us/step - loss: 17685.8828 - val_loss: 89440.0781
Epoch 80/1000
289/289 [==============================] - 0s 716us/step - loss: 21029.6035 - val_loss: 6868.3315
Epoch 81/1000
289/289 [==============================] - 0s 777us/step - loss: 19354.2363 - val_loss: 22005.6074
Epoch 82/1000
289/289 [==============================] - 0s 939us/step - loss: 24778.8652 - val_loss: 9825.4619
Ep

289/289 [==============================] - 0s 699us/step - loss: 4607.2720 - val_loss: 6634.4575
Epoch 148/1000
289/289 [==============================] - 0s 756us/step - loss: 5595.3608 - val_loss: 5961.6587
Epoch 149/1000
289/289 [==============================] - 0s 748us/step - loss: 27815.9180 - val_loss: 2171.8884
Epoch 150/1000
289/289 [==============================] - 0s 908us/step - loss: 839.6008 - val_loss: 649.4758
Epoch 151/1000
289/289 [==============================] - 0s 771us/step - loss: 1331.8673 - val_loss: 626.0139
Epoch 152/1000
289/289 [==============================] - 0s 771us/step - loss: 1933.7330 - val_loss: 15161.0801
Epoch 153/1000
289/289 [==============================] - 0s 829us/step - loss: 5215.1392 - val_loss: 759.6946
Epoch 154/1000
289/289 [==============================] - 0s 734us/step - loss: 3638.9631 - val_loss: 25198.9844
Epoch 155/1000
289/289 [==============================] - 0s 725us/step - loss: 5058.0923 - val_loss: 7355.1499
Epoch 15

...

315/315 [==============================] - 0s 833us/step - loss: 381.2236 - val_loss: 393.8194
Epoch 676/1000
315/315 [==============================] - 0s 962us/step - loss: 380.3279 - val_loss: 394.4823
Epoch 677/1000
315/315 [==============================] - 0s 914us/step - loss: 381.0431 - val_loss: 399.0356
Epoch 678/1000
315/315 [==============================] - 0s 949us/step - loss: 381.1055 - val_loss: 394.3323
Epoch 679/1000
315/315 [==============================] - 0s 873us/step - loss: 380.7079 - val_loss: 397.1339
Epoch 680/1000
315/315 [==============================] - 0s 925us/step - loss: 380.4555 - val_loss: 393.8281
Epoch 681/1000
315/315 [==============================] - 0s 903us/step - loss: 379.9278 - val_loss: 393.6232
Epoch 682/1000
315/315 [==============================] - 0s 855us/step - loss: 379.6437 - val_loss: 394.1960
Epoch 683/1000
315/315 [==============================] - 0s 862us/step - loss: 381.4026 - val_loss: 396.7133
Epoch 684/1000
315/315 [=

Epoch 750/1000
315/315 [==============================] - 0s 851us/step - loss: 380.8036 - val_loss: 408.2419
Epoch 751/1000
315/315 [==============================] - 0s 846us/step - loss: 380.4458 - val_loss: 393.6985
Epoch 752/1000
315/315 [==============================] - 0s 897us/step - loss: 380.7752 - val_loss: 394.7069
Epoch 753/1000
315/315 [==============================] - 0s 901us/step - loss: 380.0490 - val_loss: 393.7256
Epoch 754/1000
315/315 [==============================] - 0s 976us/step - loss: 380.7499 - val_loss: 394.0375
Epoch 755/1000
315/315 [==============================] - 0s 1ms/step - loss: 380.4152 - val_loss: 393.4211
Epoch 756/1000
315/315 [==============================] - 0s 993us/step - loss: 381.0882 - val_loss: 401.1212
Epoch 757/1000
315/315 [==============================] - 0s 962us/step - loss: 380.5895 - val_loss: 400.0974
Epoch 758/1000
315/315 [==============================] - 0s 931us/step - loss: 381.7116 - val_loss: 394.7275
Epoch 759/10

315/315 [==============================] - 1s 2ms/step - loss: 380.4994 - val_loss: 394.1856
Epoch 826/1000
315/315 [==============================] - 1s 3ms/step - loss: 381.0647 - val_loss: 394.1466
Epoch 827/1000
315/315 [==============================] - 1s 2ms/step - loss: 380.9471 - val_loss: 402.9354
Epoch 828/1000
315/315 [==============================] - 1s 3ms/step - loss: 379.7070 - val_loss: 393.4176
Epoch 829/1000
315/315 [==============================] - 1s 2ms/step - loss: 380.7802 - val_loss: 394.3173
Epoch 830/1000
315/315 [==============================] - 1s 2ms/step - loss: 380.0923 - val_loss: 395.2868
Epoch 831/1000
315/315 [==============================] - 1s 3ms/step - loss: 380.5560 - val_loss: 398.8377
Epoch 832/1000
315/315 [==============================] - 1s 3ms/step - loss: 380.4716 - val_loss: 393.5273
Epoch 833/1000
315/315 [==============================] - 1s 3ms/step - loss: 380.6982 - val_loss: 393.8778
Epoch 834/1000
315/315 [===================

315/315 [==============================] - 0s 887us/step - loss: 379.9522 - val_loss: 397.9109
Epoch 901/1000
315/315 [==============================] - 0s 909us/step - loss: 380.5001 - val_loss: 393.5387
Epoch 902/1000
315/315 [==============================] - 0s 911us/step - loss: 381.1636 - val_loss: 394.2971
Epoch 903/1000
315/315 [==============================] - 0s 899us/step - loss: 380.8484 - val_loss: 393.9597
Epoch 904/1000
315/315 [==============================] - 0s 946us/step - loss: 381.0553 - val_loss: 396.5442
Epoch 905/1000
315/315 [==============================] - 0s 873us/step - loss: 382.1824 - val_loss: 394.8016
Epoch 906/1000
315/315 [==============================] - 0s 914us/step - loss: 380.0180 - val_loss: 393.3888
Epoch 907/1000
315/315 [==============================] - 0s 941us/step - loss: 379.9915 - val_loss: 394.2185
Epoch 908/1000
315/315 [==============================] - 0s 897us/step - loss: 380.6626 - val_loss: 402.3841
Epoch 909/1000
315/315 [=

315/315 [==============================] - 0s 898us/step - loss: 379.9427 - val_loss: 402.4526
Epoch 975/1000
315/315 [==============================] - 0s 925us/step - loss: 381.3503 - val_loss: 402.7602
Epoch 976/1000
315/315 [==============================] - 0s 943us/step - loss: 380.2381 - val_loss: 399.4900
Epoch 977/1000
315/315 [==============================] - 0s 946us/step - loss: 379.2683 - val_loss: 396.8351
Epoch 978/1000
315/315 [==============================] - 0s 893us/step - loss: 380.1035 - val_loss: 393.5959
Epoch 979/1000
315/315 [==============================] - 0s 978us/step - loss: 381.1729 - val_loss: 398.5750
Epoch 980/1000
315/315 [==============================] - 0s 894us/step - loss: 382.1765 - val_loss: 393.6116
Epoch 981/1000
315/315 [==============================] - 0s 1ms/step - loss: 379.8884 - val_loss: 399.9914
Epoch 982/1000
315/315 [==============================] - 0s 865us/step - loss: 380.9037 - val_loss: 395.4620
Epoch 983/1000
315/315 [===

In [20]:
mean_rmse = np.mean(rmse_list)          
mean_rmse

20.168720017035618

In [21]:
rmse_list

[11.101412828338793,
 28.11980326454524,
 15.857607314085183,
 21.598058026456492,
 12.737806158141042,
 25.56834626242732,
 20.239426744496644,
 13.13355844753267,
 26.146129076490737,
 27.185052047842035]

In [24]:
mean_mape = np.mean(mape_list)          
mean_mape

0.5616395260546081

In [23]:
mape_list

[0.6905492667002636,
 0.44198048430242154,
 0.6008417643018131,
 0.4374656701571717,
 0.575685004499461,
 0.5505790877083362,
 0.5274268689222783,
 0.7485715150094702,
 0.5191001726361589,
 0.5241954263087066]